# TensorFlow2.0教程-使用RNN实现文本分类

In [5]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import tensorflow.keras as keras
print(tf.__version__)


2.0.0-alpha0


## 1.使用tensorflow_datasets 构造输入数据

In [7]:
!pip install -q tensorflow_datasets

spacy 2.0.18 has requirement numpy>=1.15.0, but you'll have numpy 1.14.3 which is incompatible.
plotnine 0.5.1 has requirement matplotlib>=3.0.0, but you'll have matplotlib 2.2.2 which is incompatible.
plotnine 0.5.1 has requirement pandas>=0.23.4, but you'll have pandas 0.23.0 which is incompatible.
neo4j-driver 1.6.2 has requirement neotime==1.0.0, but you'll have neotime 1.7.2 which is incompatible.
mizani 0.5.3 has requirement pandas>=0.23.4, but you'll have pandas 0.23.0 which is incompatible.
fastai 0.7.0 has requirement torch<0.4, but you'll have torch 0.4.1 which is incompatible.
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [8]:
import tensorflow_datasets as tfds
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True,
                         as_supervised=True) 

Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...:   0%|          | 0/1 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Size...:  74%|███████▍  | 59/80 [2:00:33<29:19, 83.80s/ MiB]


Dl Size...: 100%|██████████| 80/80 [2:24:37<00:00, 52.36s/ MiB]
0 examples [00:00, ? examples/s]

25000 examples [00:25, 975.34 examples/s] 
Shuffling...:   0%|          | 0/10 [00:00<?, ? shard/s]WARNING: Logging before flag parsing goes to stderr.
W0408 01:56:15.224409 140157398619904 deprecation.py:323] From /home/czy/anaconda3/envs/tf2_0/lib/python3.6/site-packages/tensorflow_datasets/core/file_format_adapter.py:249: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`

Reading...: 0 examples [00:00, ? examples/s]
Reading...: 2500 examples [00:00, 60539.24 examples/s]
Writing...: 100%|██████████| 2500/2500 [00:00<00:00, 93611.99 examples/s]
Reading...: 0 examples [00:00, ? examples/s]
Reading...: 2500 examples [00:00, 180642.58 examples/s]
Shuffling...:  20%|██        | 2/10 [00:00<00:00, 18.33 shard/s]xamples/s]
Reading...: 0 examples [00:00, ? examples/s]
Reading...: 2500 examples [00:00, 263434.83 examples/s]
Writing...: 100%|███